# JS - PLS XGB - Submission

In [ ]:
import joblib
import json
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

In [ ]:
# load the model
model_folder = os.path.join(os.pardir, "input", "js-pls-xgb-training")

pp = joblib.load(os.path.join(model_folder, "preprocessor.pkl"))

model = xgb.XGBClassifier()
model.load_model(os.path.join(model_folder, "model.xgb"))

# get the optimal threshold
utility_folder = os.path.join(os.pardir, "input", "js-pls-xgb-utility")

with open(os.path.join(utility_folder, "threshold.json")) as f:
    threshold_json = json.loads(f.read())

threshold = float(threshold_json["threshold"])

In [ ]:
# function to return predictions for the current trade
def predict(sample):
    X = sample.to_numpy()
    X = pp.transform(X)
    prob = model.predict_proba(X)
    return 1 if prob[0, 1] > threshold else 0

In [ ]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

# initialize things with first sample
sample, pred = next(iter_test)
features = [c for c in sample.columns if "feature" in c]
pred.action = predict(sample[features])
env.predict(pred)

# loop over remaining samples
for (sample, pred) in iter_test:
    pred.action = predict(sample[features]) if sample["weight"].item() > 0.0 else 0
    env.predict(pred)